In [1]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/surya/CV_Assignment/Final_Course_Project/Color-Transfer-between-Images-master')
#os.chdir('/content/drive/My Drive/CV/FinalProject')

Mounted at /content/drive


In [7]:
import numpy as np
import cv2
import shutil
from google.colab import files
from IPython.display import display

# Function to load an image file from a given path
def load_images(source_path, target_path):
  # Load source image and convert to LAB color space
  source_img = cv2.imread(source_path)
  source_img = cv2.cvtColor(source_img, cv2.COLOR_BGR2LAB)
  # Load target image and convert to LAB color space
  target_img = cv2.imread(target_path)
  target_img = cv2.cvtColor(target_img, cv2.COLOR_BGR2LAB)
  return source_img, target_img

# Function to compute the mean and standard deviation for an image
def compute_image_stats(image):
  mean, std_dev = cv2.meanStdDev(image)
  mean = np.hstack(np.around(mean, 2))
  std_dev = np.hstack(np.around(std_dev, 2))
  return mean, std_dev

# Function to apply color transfer from source to target images
def apply_color_transfer(source_files, target_files):
  # Delete the contents of the 'result' folder before starting
  result_folder = 'result'
  if os.path.exists(result_folder):
    shutil.rmtree(result_folder)
  os.makedirs(result_folder)

  # Process each pair of source and target images
  for index in range(len(source_files)):
    print(f"Processing image pair {index + 1}...")
    source_img, target_img = load_images(source_files[index], target_files[index])
    source_mean, source_std_dev = compute_image_stats(source_img)
    target_mean, target_std_dev = compute_image_stats(target_img)

    # Get the dimensions of the source image
    height, width, channels = source_img.shape
    for i in range(height):
      for j in range(width):
        for k in range(channels):
          # Calculate the new pixel value using color transfer formula
          pixel_value = source_img[i, j, k]
          pixel_value = ((pixel_value - source_mean[k]) * (target_std_dev[k] / source_std_dev[k])) + target_mean[k]
          # Round the pixel value
          pixel_value = round(pixel_value)
          # Clamp the pixel value within the valid range
          pixel_value = max(0, min(255, pixel_value))
          source_img[i, j, k] = pixel_value

    # Convert the source image back to BGR color space
    source_img = cv2.cvtColor(source_img, cv2.COLOR_LAB2BGR)
    result_path = f'{result_folder}/result_{index + 1}.bmp'
    # Save the resulting image
    cv2.imwrite(result_path, source_img)
    print(f"Result saved to {result_path}")
    # Download the resulting image
    display(files.download(result_path))

# Main function to handle user input and initiate color transfer
def run_program():
  print("Please upload the source images:")
  source_files_dict = files.upload()
  print("Please upload the target images:")
  target_files_dict = files.upload()

  # Save uploaded files to the source and target folders
  source_files = []
  for filename, data in source_files_dict.items():
    file_path = f'source/{filename}'
    with open(file_path, 'wb') as file:
      file.write(data)
    source_files.append(file_path)

  target_files = []
  for filename, data in target_files_dict.items():
    file_path = f'target/{filename}'
    with open(file_path, 'wb') as file:
      file.write(data)
    target_files.append(file_path)

  # Validate that the number of source and target files are equal
  if len(source_files) != len(target_files):
    print("Mismatch in the number of source and target files. Please try again.")
    return

  # Perform color transfer
  apply_color_transfer(source_files, target_files)

# Run the program
run_program()


Please upload the source images:


Saving Source.jpg to Source.jpg
Please upload the target images:


Saving Target.jpg to Target.jpg
Processing image pair 1...
Result saved to result/result_1.bmp


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

None

In [5]:
# Function to delete images in a given folder
def delete_images(folder):
  # Check if the folder exists
  if os.path.exists(folder):
    # Remove all files in the folder
    for file_name in os.listdir(folder):
      file_path = os.path.join(folder, file_name)
      # Check if the file is an image (e.g., .bmp, .jpg, .jpeg, .png)
      if os.path.isfile(file_path) and file_name.endswith(('.bmp', '.jpg', '.jpeg', '.png')):
        os.remove(file_path)
        print(f"Deleted file: {file_path}")

# Path to the base directory containing the source, target, and result folders
base_path = '/content/drive/MyDrive/surya/CV_Assignment/Final_Course_Project/Color-Transfer-between-Images-master'

# Full paths to the source, target, and result directories
source_path = os.path.join(base_path, 'source')
target_path = os.path.join(base_path, 'target')
result_path = os.path.join(base_path, 'result')

# Delete images from the source, target, and result directories
delete_images(source_path)
delete_images(target_path)
delete_images(result_path)

base_path = '/content/drive/MyDrive/surya/CV_Assignment/Final_Course_Project/Color-Transfer-between-Images-master'
delete_images(base_path)

print("All images deleted from source, target, and result directories.")


Deleted file: /content/drive/MyDrive/surya/CV_Assignment/Final_Course_Project/Color-Transfer-between-Images-master/source/Source.jpg
Deleted file: /content/drive/MyDrive/surya/CV_Assignment/Final_Course_Project/Color-Transfer-between-Images-master/target/Target.jpg
Deleted file: /content/drive/MyDrive/surya/CV_Assignment/Final_Course_Project/Color-Transfer-between-Images-master/result/result_1.bmp
Deleted file: /content/drive/MyDrive/surya/CV_Assignment/Final_Course_Project/Color-Transfer-between-Images-master/Source.jpg
Deleted file: /content/drive/MyDrive/surya/CV_Assignment/Final_Course_Project/Color-Transfer-between-Images-master/Target.jpg
All images deleted from source, target, and result directories.


In [ ]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

# Computes the mean and standard deviation of each channel (L, a, b) in an image
def compute_stats(image):
  l_channel, a_channel, b_channel = cv2.split(image)
  l_mean, l_std_dev = l_channel.mean(), l_channel.std()
  a_mean, a_std_dev = a_channel.mean(), a_channel.std()
  b_mean, b_std_dev = b_channel.mean(), b_channel.std()
  return l_mean, l_std_dev, a_mean, a_std_dev, b_mean, b_std_dev

# Transfers colors from source image to destination image
def apply_color_transfer(source_image, destination_image):
  # Convert source and destination images to LAB color space
  source_lab = cv2.cvtColor(source_image, cv2.COLOR_BGR2LAB).astype("float32")
  destination_lab = cv2.cvtColor(destination_image, cv2.COLOR_BGR2LAB).astype("float32")

  # Get statistics of source and destination images
  source_stats = compute_stats(source_lab)
  destination_stats = compute_stats(destination_lab)

  l, a, b = cv2.split(destination_lab)

  # Apply color transfer by normalizing and scaling each channel
  l = ((l - destination_stats[0]) * (destination_stats[1] / source_stats[1])) + source_stats[0]
  a = ((a - destination_stats[2]) * (destination_stats[3] / source_stats[3])) + source_stats[2]
  b = ((b - destination_stats[4]) * (destination_stats[5] / source_stats[5])) + source_stats[4]

  # Ensure the channel values are within the valid range [0, 255]
  l, a, b = np.clip(l, 0, 255), np.clip(a, 0, 255), np.clip(b, 0, 255)

  # Merge the channels back and convert to BGR color space
  final_image = cv2.merge([l, a, b])
  final_image = cv2.cvtColor(final_image.astype("uint8"), cv2.COLOR_LAB2BGR)

  return final_image

# Function to display images using cv2_imshow
def display_image(title, img):
  print(f"{title}:")
  cv2_imshow(img)

# Paths to the source, destination, and output images
source_img_path = "/content/drive/MyDrive/surya/CV_Assignment/Final_Course_Project/Source.jpg"
destination_img_path = "/content/drive/MyDrive/surya/CV_Assignment/Final_Course_Project/Target.jpg"
output_img_path = "/content/drive/MyDrive/surya/CV_Assignment/Final_Course_Project/Dest.jpg"

# Read source and destination images
source_image = cv2.imread(source_img_path)
destination_image = cv2.imread(destination_img_path)

# Perform color transfer and save the result
result_image = apply_color_transfer(destination_image , source_image )
cv2.imwrite(output_img_path, result_image)

# Display the images
display_image("Source", source_image)
display_image("Destination", destination_image)
display_image("Transferred", result_image)


In [6]:
import cv2
import numpy as np

# Load the images from the specified file paths
src_img = cv2.imread("/content/drive/MyDrive/surya/CV_Assignment/Final_Course_Project/Source.jpg")
tgt_img = cv2.imread("/content/drive/MyDrive/surya/CV_Assignment/Final_Course_Project/Target.jpg")
result_img = cv2.imread("/content/drive/MyDrive/surya/CV_Assignment/Final_Course_Project/Dest.jpg")

# Transform the images to the HSV color model
src_img_hsv = cv2.cvtColor(src_img, cv2.COLOR_BGR2HSV)
tgt_img_hsv = cv2.cvtColor(tgt_img, cv2.COLOR_BGR2HSV)
result_img_hsv = cv2.cvtColor(result_img, cv2.COLOR_BGR2HSV)

# Calculate the histograms for each image in HSV color space
hist_src = cv2.calcHist([src_img_hsv], [0, 1, 2], None, [50, 50, 50], [0, 180, 0, 256, 0, 256])
hist_tgt = cv2.calcHist([tgt_img_hsv], [0, 1, 2], None, [50, 50, 50], [0, 180, 0, 256, 0, 256])
hist_result = cv2.calcHist([result_img_hsv], [0, 1, 2], None, [50, 50, 50], [0, 180, 0, 256, 0, 256])

# Normalize the histograms to prepare them for comparison
norm_hist_src = cv2.normalize(hist_src, hist_src).flatten()
norm_hist_tgt = cv2.normalize(hist_tgt, hist_tgt).flatten()
norm_hist_result = cv2.normalize(hist_result, hist_result).flatten()

# Compare the histograms using a correlation method (cv2.HISTCMP_CORREL)
corr_src_tgt = cv2.compareHist(norm_hist_src, norm_hist_tgt, cv2.HISTCMP_CORREL)
corr_src_result = cv2.compareHist(norm_hist_src, norm_hist_result, cv2.HISTCMP_CORREL)

# Output the correlation values to the console
print("Similarity between source and target images:", corr_src_tgt)
print("Similarity between source and result images:", corr_src_result)


Similarity between source and target images: -0.0012995733343451673
Similarity between source and result images: -0.00078170889540618


In [ ]:
import cv2
import numpy as np

# Load the source and target images.
src_img = cv2.imread("/content/drive/MyDrive/surya/CV_Assignment/Final_Course_Project/Source.jpg")
tgt_img = cv2.imread("/content/drive/MyDrive/surya/CV_Assignment/Final_Course_Project/Target.jpg")
result_img = cv2.imread("/content/drive/MyDrive/surya/CV_Assignment/Final_Course_Project/Dest.jpg")

source_image = cv2.cvtColor(src_img, cv2.COLOR_BGR2Lab)
target_image = cv2.cvtColor(tgt_img, cv2.COLOR_BGR2Lab)
results_image = cv2.cvtColor(result_img, cv2.COLOR_BGR2Lab)

# Evaluate the MSE.
mse = np.mean((source_image - results_image) ** 2)

# Print the MSE.
print("MSE:", mse)

MSE: 101.09176336888983
